# 🕒 REQ-3: Enforce Retention Policies

> "As a data steward, I need to enforce retention policies on datasets.  
> Datasets nearing expiry trigger alerts; expired datasets are flagged for review."

This notebook demonstrates:
- How `retention_policy` metadata is stored in Unity Catalog
- How to simulate audit logic to identify datasets nearing or exceeding retention
- Opportunities to link this with lifecycle automation or alerts

Catalog: `unity_demo`  
Schema: `governance_lab`  


In [ ]:
# Retention metadata simulation
import datetime

# Example: simulate retention timestamps as if the tables had a "created_at" field
table_created_dates = {
    "employee_records": datetime.date(2019, 7, 20),
    "customer_orders": datetime.date(2022, 3, 15),
}

today = datetime.date.today()

for tbl, created in table_created_dates.items():
    props = spark.sql(f"SHOW TBLPROPERTIES unity_demo.governance_lab.{tbl}").rdd.collectAsMap()
    policy = props.get("retention_policy", "not_set")
    if "y" in policy:
        years = int(policy.replace("y", ""))
        expiry = created + datetime.timedelta(days=365 * years)
        print(f"🗂️ Table: {tbl} | Created: {created} | Policy: {policy} | Expires: {expiry}")
        if today > expiry:
            print("⚠️ Dataset has expired!")
        elif (expiry - today).days < 180:
            print("⏳ Dataset near expiry (within 6 months).")
        else:
            print("✅ Retention is valid.")
    else:
        print(f"❌ Invalid or missing policy on {tbl}")
